# Week 2: Lab Solutions - Q-Learning for Nim

---

This notebook contains complete solutions with detailed explanations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random

np.random.seed(42)
random.seed(42)

## Nim Environment (Provided)

In [ ]:
class NimGame:
    """Nim game environment."""
    
    def __init__(self, initial_piles):
        self.initial_piles = list(initial_piles)
        self.reset()
    
    def reset(self):
        self.piles = list(self.initial_piles)
        self.done = False
        self.current_player = 0
        return self.get_state()
    
    def get_state(self):
        return tuple(self.piles)
    
    def get_valid_actions(self):
        actions = []
        for pile_idx, pile_size in enumerate(self.piles):
            for num_remove in range(1, pile_size + 1):
                actions.append((pile_idx, num_remove))
        return actions
    
    def step(self, action):
        pile_idx, num_remove = action
        self.piles[pile_idx] -= num_remove
        
        if sum(self.piles) == 0:
            self.done = True
            return self.get_state(), -1, True
        
        self.current_player = 1 - self.current_player
        return self.get_state(), 0, False
    
    def render(self):
        print("\n" + "=" * 30)
        print(f"Player {self.current_player + 1}'s turn")
        print("=" * 30)
        for i, pile in enumerate(self.piles):
            objects = "●" * pile if pile > 0 else "(empty)"
            print(f"Pile {i}: {objects}  ({pile})")
        print()


def random_agent(game):
    return random.choice(game.get_valid_actions())

---

## Part 4 Solution: Setting Hyperparameters

### Explanation of Hyperparameters

**α (alpha) - Learning Rate = 0.1**
- Controls how much new information overrides old information
- Too high (e.g., 0.9): Unstable learning, values fluctuate wildly
- Too low (e.g., 0.01): Very slow learning
- 0.1 is a good default that balances speed and stability

**γ (gamma) - Discount Factor = 0.9**
- Controls importance of future rewards vs immediate rewards
- γ = 1.0: Future rewards are equally important
- γ = 0: Only immediate rewards matter
- 0.9 gives good weight to future while still preferring sooner rewards

**ε (epsilon) - Exploration Rate = 0.3**
- Probability of taking a random action
- Too low: Agent gets stuck in local optima
- Too high: Too much random exploration, slow convergence
- 0.3 during training allows good exploration; use 0 during evaluation

---

## Part 5 Solution: Q-Learning Update

### The Complete Implementation

In [ ]:
class QLearningAgent:
    """Q-Learning agent for Nim - COMPLETE SOLUTION."""
    
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.3):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = defaultdict(float)
    
    def get_q_value(self, state, action):
        return self.Q[(state, action)]
    
    def get_best_action(self, state, valid_actions):
        q_values = [self.get_q_value(state, a) for a in valid_actions]
        max_q = max(q_values)
        best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_q]
        return random.choice(best_actions)
    
    def select_action(self, game, training=True):
        state = game.get_state()
        valid_actions = game.get_valid_actions()
        
        if training and random.random() < self.epsilon:
            return random.choice(valid_actions)
        else:
            return self.get_best_action(state, valid_actions)
    
    def update(self, state, action, reward, next_state, next_valid_actions, done):
        """
        Q-Learning update rule:
        Q(s,a) <- Q(s,a) + alpha * [reward + gamma * max_a' Q(s',a') - Q(s,a)]
        """
        # Step 1: Get current Q-value
        current_q = self.get_q_value(state, action)
        
        # Step 2: Calculate target
        if done:
            # SOLUTION: Terminal state - just use reward
            # No future states, so no discounted future value
            target = reward
        else:
            # SOLUTION: Get max Q-value for next state
            next_q_values = [self.get_q_value(next_state, a) for a in next_valid_actions]
            max_next_q = max(next_q_values)  # SOLUTION: Take the maximum
            
            # SOLUTION: Target = reward + discounted future value
            target = reward + self.gamma * max_next_q
        
        # Step 3: Update Q-value
        # SOLUTION: Apply the Q-learning update
        new_q = current_q + self.alpha * (target - current_q)
        
        self.Q[(state, action)] = new_q

### Detailed Explanation of the Update

Let's trace through the update step by step:

**Example: Non-terminal state**
- State s = (3, 4) - piles have 3 and 4 objects
- Action a = (0, 1) - remove 1 from pile 0
- Reward r = 0 (game continues)
- Next state s' = (2, 4)

```
current_q = Q[(3,4), (0,1)]  # e.g., 0.2
max_next_q = max(Q[(2,4), a'] for all valid a')  # e.g., 0.5
target = 0 + 0.9 * 0.5 = 0.45
new_q = 0.2 + 0.1 * (0.45 - 0.2) = 0.2 + 0.025 = 0.225
```

**Example: Terminal state (agent loses)**
- State s = (1,) - one object left
- Action a = (0, 1) - take the last object
- Reward r = -1 (agent loses)
- Done = True

```
current_q = Q[(1,), (0,1)]  # e.g., 0
target = -1  # No future, just reward
new_q = 0 + 0.1 * (-1 - 0) = -0.1
```

Over many updates, Q-values converge to reflect true action values!

In [ ]:
# Test the implementation
agent = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.3)

# Test case 1: Terminal state with reward
agent.update(
    state=(1,),
    action=(0, 1),
    reward=1.0,
    next_state=(0,),
    next_valid_actions=[],
    done=True
)
result1 = agent.get_q_value((1,), (0, 1))
print(f"Test 1 - Terminal state:")
print(f"  Q((1,), (0,1)) = {result1:.3f}")
print(f"  Expected: 0.100 ✓" if abs(result1 - 0.1) < 0.001 else f"  Expected: 0.100 ✗")

# Test case 2: Non-terminal state
agent.Q[((1,), (0, 1))] = 0.5  # Set known value
agent.update(
    state=(2,),
    action=(0, 1),
    reward=0.0,
    next_state=(1,),
    next_valid_actions=[(0, 1)],
    done=False
)
result2 = agent.get_q_value((2,), (0, 1))
expected2 = 0 + 0.1 * (0 + 0.9 * 0.5 - 0)
print(f"\nTest 2 - Non-terminal state:")
print(f"  Q((2,), (0,1)) = {result2:.3f}")
print(f"  Expected: {expected2:.3f} ✓" if abs(result2 - expected2) < 0.001 else f"  Expected: {expected2:.3f} ✗")

---

## Training Loop and Evaluation

In [ ]:
def train_agent(agent, game, opponent, n_episodes=5000, verbose_every=1000):
    """Train Q-learning agent by playing against opponent."""
    wins = 0
    win_rates = []
    
    for episode in range(n_episodes):
        state = game.reset()
        agent_history = []
        
        while not game.done:
            if game.current_player == 0:
                action = agent.select_action(game, training=True)
                agent_history.append((state, action))
            else:
                action = opponent(game)
            
            next_state, reward, done = game.step(action)
            
            if game.current_player == 1 or done:
                if len(agent_history) > 0:
                    s, a = agent_history[-1]
                    
                    if done:
                        if game.current_player == 1:
                            agent_reward = -1
                        else:
                            agent_reward = 1
                    else:
                        agent_reward = 0
                    
                    next_valid = game.get_valid_actions() if not done else []
                    agent.update(s, a, agent_reward, next_state, next_valid, done)
            
            state = next_state
        
        if game.current_player == 1:
            wins += 1
        
        if (episode + 1) % 100 == 0:
            win_rates.append(wins / (episode + 1))
        
        if verbose_every and (episode + 1) % verbose_every == 0:
            print(f"Episode {episode + 1}: Win Rate = {wins/(episode+1)*100:.1f}%")
    
    return win_rates


def evaluate_agent(agent, game, opponent, n_games=1000):
    """Evaluate agent without exploration."""
    wins = 0
    
    for _ in range(n_games):
        state = game.reset()
        
        while not game.done:
            if game.current_player == 0:
                action = agent.select_action(game, training=False)
            else:
                action = opponent(game)
            state, reward, done = game.step(action)
        
        if game.current_player == 1:
            wins += 1
    
    return wins / n_games

In [ ]:
# Train on 2-pile game
print("Training on [3, 4] piles...\n")
agent = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.3)
game = NimGame([3, 4])

win_rates = train_agent(agent, game, random_agent, n_episodes=5000)

# Evaluate
final_win_rate = evaluate_agent(agent, game, random_agent, n_games=1000)
print(f"\nFinal win rate: {final_win_rate*100:.1f}%")
print(f"Q-table size: {len(agent.Q)} entries")

In [ ]:
# Plot learning curve
plt.figure(figsize=(10, 5))
plt.plot(np.arange(100, 5001, 100), win_rates)
plt.xlabel('Episodes')
plt.ylabel('Win Rate')
plt.title('Q-Learning Agent Win Rate Over Training')
plt.axhline(y=0.5, color='r', linestyle='--', label='Random baseline (50%)', alpha=0.7)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 9 Solution: Multiple Pile Configurations

In [ ]:
pile_configs = [
    [3],
    [3, 4],
    [3, 4, 5],
    [2, 3, 4, 5],
]

results = {}

for piles in pile_configs:
    print(f"\nTraining on piles: {piles}")
    print("-" * 40)
    
    agent = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.3)
    game = NimGame(piles)
    
    win_rates = train_agent(agent, game, random_agent, n_episodes=10000, verbose_every=5000)
    final_win_rate = evaluate_agent(agent, game, random_agent, n_games=1000)
    
    results[tuple(piles)] = {
        'agent': agent,
        'win_rates': win_rates,
        'final_rate': final_win_rate,
        'q_size': len(agent.Q)
    }
    
    print(f"Final win rate: {final_win_rate*100:.1f}%")
    print(f"Q-table size: {len(agent.Q)} entries")

In [ ]:
# Summary and visualization
print("\n" + "=" * 60)
print("SUMMARY: Performance by Complexity")
print("=" * 60)

for piles, data in results.items():
    print(f"Piles {str(list(piles)):15} | Win Rate: {data['final_rate']*100:5.1f}% | Q-table: {data['q_size']:6} entries")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
labels = [str(list(p)) for p in results.keys()]
win_rates_list = [data['final_rate'] * 100 for data in results.values()]
plt.bar(labels, win_rates_list, color='steelblue')
plt.axhline(y=50, color='r', linestyle='--', label='Random baseline')
plt.xlabel('Pile Configuration')
plt.ylabel('Win Rate (%)')
plt.title('Agent Performance by Complexity')
plt.legend()

plt.subplot(1, 2, 2)
q_sizes = [data['q_size'] for data in results.values()]
plt.bar(labels, q_sizes, color='coral')
plt.xlabel('Pile Configuration')
plt.ylabel('Q-table Size')
plt.title('State Space Size by Complexity')

plt.tight_layout()
plt.show()

---

## Part 10 Solutions: Analysis Questions

### 1. Q-table Size Growth

**Observation:** The Q-table grows exponentially with the number of piles.

**Why:**
- For a single pile of size n, there are n possible states
- For two piles of sizes n and m, there are n × m states
- For k piles, state space is approximately ∏(sizes)
- Additionally, each state has multiple actions

This exponential growth is called the **curse of dimensionality** and is why tabular RL doesn't scale to large problems.

### 2. Win Rate vs Complexity

**Observation:** Win rate generally decreases with more piles (though still beats random).

**Why:**
1. **Larger state space:** With more states, the agent needs more experience to visit each state enough times
2. **More strategic depth:** The optimal Nim strategy involves XOR operations across all piles
3. **Exploration inefficiency:** With more actions per state, random exploration is less effective

### 3. No Exploration (ε = 0)

With ε = 0, the agent:
- Only takes actions it thinks are best (from the start)
- Never explores alternative strategies
- Gets stuck in local optima
- Many state-action pairs never get visited

### 4. Agent Strategy

The agent learns to:
- Avoid leaving a single object (losing position)
- Aim for symmetric positions like (1,1) where it can mirror opponent
- The optimal strategy involves XOR (Nim-sum), which the agent approximates

---

## Bonus 1 Solution: Self-Play Training

In [ ]:
def train_self_play(initial_piles, n_episodes=10000, verbose_every=2000):
    """Train two agents against each other."""
    game = NimGame(initial_piles)
    
    agent_0 = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.3)
    agent_1 = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.3)
    
    wins = [0, 0]
    
    for episode in range(n_episodes):
        state = game.reset()
        history = {0: [], 1: []}
        
        while not game.done:
            current = game.current_player
            agent = agent_0 if current == 0 else agent_1
            
            action = agent.select_action(game, training=True)
            history[current].append((state, action))
            
            next_state, reward, done = game.step(action)
            state = next_state
        
        # Determine winner (current_player just lost)
        loser = 1 - game.current_player
        winner = game.current_player
        wins[winner] += 1
        
        # Update both agents
        for player_id, agent in [(0, agent_0), (1, agent_1)]:
            player_history = history[player_id]
            
            for i, (s, a) in enumerate(player_history):
                if i == len(player_history) - 1:
                    r = 1 if player_id == winner else -1
                    agent.update(s, a, r, state, [], True)
                else:
                    next_s = player_history[i + 1][0]
                    game.piles = list(next_s)
                    next_valid = game.get_valid_actions()
                    agent.update(s, a, 0, next_s, next_valid, False)
        
        if verbose_every and (episode + 1) % verbose_every == 0:
            print(f"Episode {episode + 1}: Agent 0 wins: {wins[0]}, Agent 1 wins: {wins[1]}")
    
    return agent_0, agent_1


print("Training with self-play...\n")
agent_sp_0, agent_sp_1 = train_self_play([3, 4], n_episodes=10000)

# Evaluate self-play agent against random
game = NimGame([3, 4])
sp_win_rate = evaluate_agent(agent_sp_0, game, random_agent, n_games=1000)
print(f"\nSelf-play agent vs Random: {sp_win_rate*100:.1f}%")

---

## Bonus 2 Solution: Epsilon Decay

In [ ]:
class DecayingEpsilonAgent(QLearningAgent):
    """Q-Learning with decaying exploration."""
    
    def __init__(self, alpha=0.1, gamma=0.9, epsilon_start=1.0, 
                 epsilon_end=0.01, epsilon_decay=0.9995):
        super().__init__(alpha, gamma, epsilon_start)
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
    
    def decay_epsilon(self):
        """Call after each episode to decay epsilon."""
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)


def train_with_decay(agent, game, opponent, n_episodes=10000, verbose_every=2000):
    """Training loop with epsilon decay."""
    wins = 0
    win_rates = []
    epsilons = []
    
    for episode in range(n_episodes):
        state = game.reset()
        agent_history = []
        
        while not game.done:
            if game.current_player == 0:
                action = agent.select_action(game, training=True)
                agent_history.append((state, action))
            else:
                action = opponent(game)
            
            next_state, reward, done = game.step(action)
            
            if game.current_player == 1 or done:
                if len(agent_history) > 0:
                    s, a = agent_history[-1]
                    if done:
                        agent_reward = -1 if game.current_player == 1 else 1
                    else:
                        agent_reward = 0
                    next_valid = game.get_valid_actions() if not done else []
                    agent.update(s, a, agent_reward, next_state, next_valid, done)
            
            state = next_state
        
        if game.current_player == 1:
            wins += 1
        
        agent.decay_epsilon()
        
        if (episode + 1) % 100 == 0:
            win_rates.append(wins / (episode + 1))
            epsilons.append(agent.epsilon)
        
        if verbose_every and (episode + 1) % verbose_every == 0:
            print(f"Episode {episode + 1}: Win Rate = {wins/(episode+1)*100:.1f}%, ε = {agent.epsilon:.4f}")
    
    return win_rates, epsilons


print("Training with epsilon decay...\n")
decay_agent = DecayingEpsilonAgent()
game = NimGame([3, 4])

win_rates_decay, epsilons = train_with_decay(decay_agent, game, random_agent, n_episodes=10000)
final_decay = evaluate_agent(decay_agent, game, random_agent, n_games=1000)

print(f"\nWith decay: {final_decay*100:.1f}%")

In [ ]:
# Plot epsilon decay
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(np.arange(100, 10001, 100), epsilons)
plt.xlabel('Episodes')
plt.ylabel('Epsilon')
plt.title('Epsilon Decay Over Training')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(np.arange(100, 10001, 100), win_rates_decay, label='With decay')
plt.xlabel('Episodes')
plt.ylabel('Win Rate')
plt.title('Win Rate with Epsilon Decay')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Bonus 3 Solution: SARSA Comparison

In [ ]:
class SARSAAgent:
    """SARSA agent - on-policy TD learning."""
    
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.3):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = defaultdict(float)
    
    def get_q_value(self, state, action):
        return self.Q[(state, action)]
    
    def get_best_action(self, state, valid_actions):
        q_values = [self.get_q_value(state, a) for a in valid_actions]
        max_q = max(q_values)
        best_actions = [a for a, q in zip(valid_actions, q_values) if q == max_q]
        return random.choice(best_actions)
    
    def select_action(self, game, training=True):
        state = game.get_state()
        valid_actions = game.get_valid_actions()
        
        if training and random.random() < self.epsilon:
            return random.choice(valid_actions)
        else:
            return self.get_best_action(state, valid_actions)
    
    def update(self, state, action, reward, next_state, next_action, done):
        """
        SARSA update: uses actual next action, not max.
        Q(s,a) <- Q(s,a) + alpha * [r + gamma * Q(s',a') - Q(s,a)]
        """
        current_q = self.get_q_value(state, action)
        
        if done:
            target = reward
        else:
            next_q = self.get_q_value(next_state, next_action)
            target = reward + self.gamma * next_q
        
        self.Q[(state, action)] = current_q + self.alpha * (target - current_q)


print("SARSA vs Q-Learning Comparison")
print("="*40)
print("\nQ-Learning (off-policy): Updates toward max Q(s',a')")
print("SARSA (on-policy): Updates toward actual next action Q(s',a')")
print("\nFor deterministic games like Nim, both perform similarly.")

---

## Key Takeaways

1. **Q-learning works well for small state spaces** like Nim with few piles

2. **State space explosion** limits tabular methods - this motivates Deep Q-Networks

3. **Exploration is crucial** - ε=0 leads to poor learning

4. **Self-play** can produce stronger agents than training against random

5. **Epsilon decay** balances exploration early and exploitation later

**Next week:** We'll see how to handle large state spaces using function approximation with neural networks (DQN)!